In [1]:
import pandas as pd
import ast
import numpy as np
import scipy.stats as stats 
import seaborn as sns
import matplotlib.pyplot as plt
import os
from statsmodels.stats.multitest import multipletests

# Read the JSON blob formats

In [2]:
os.chdir('../')
os.getcwd()

'/Users/xehu/Desktop/Team Process Mapping/tpm-horse-race-modeling'

In [ ]:
df_ax = pd.DataFrame([ast.literal_eval(x.strip()) for x in open("./model_eval_cache/", "r").readlines()])

sns.pointplot(x="sample_size", y="q2", hue="model_label", data=df_ax, dodge=0.2)
plt.axhline(0, color="black", linestyle="--")
plt.xlabel("Sample size")
plt.ylabel("$Q^2$")
plt.title("$Q^2$ with HPO")

## TODO -- modify code below!

In [ ]:
def display_feature_coefficients(feature_coef_df):
	# Initialize a list to store DataFrames for each feature
	dfs = []

	# Iterate through the rows of the input DataFrame
	for feature_name, coefficients in feature_coef_df.iterrows():
		# Calculate the confidence interval without NaN values
		non_nan_coefficients = coefficients[~np.isnan(coefficients)]
		if len(non_nan_coefficients) == 0:
			# Handle the case where there are no valid coefficients
			continue

		mean_coef = non_nan_coefficients.mean()

		# Check if all coefficients in the row are the same
		if len(coefficients.unique()) == 1:
			# If all coefficients are the same, set the lower and upper CI to the mean
			confidence_interval = (mean_coef, mean_coef)
		else:
			std_error = non_nan_coefficients.sem()
			confidence_interval = stats.t.interval(0.95, len(non_nan_coefficients) - 1, loc=mean_coef, scale=std_error)

		# Create a DataFrame for the summary data
		temp_df = pd.DataFrame({
			"Feature": [feature_name],
			"Mean": [mean_coef],
			"Lower_CI": [confidence_interval[0]],
			"Upper_CI": [confidence_interval[1]]
		})

		# Append the temporary DataFrame to the list
		dfs.append(temp_df)

	# Concatenate all the DataFrames in the list into the final summary DataFrame
	summary_df = pd.concat(dfs, ignore_index=True)

	return summary_df